In [1]:
import os
os.environ['GOOGLE_API_KEY'] = "Please insert your google studio api key here"

In [2]:
#Install libraries
!pip install langchain langchain-community faiss-cpu tiktoken google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 --upgrade youtube-transcript-api \
langchain-google-genai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [3]:
#Import libraries
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

**Step 1a - Indexing (Document Ingestion)**

In [6]:
video_id = 'g1Z4eSq6AAw' # only the ID, not full URL

try:
  transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=['en','hi']) # If you don’t care which language, this returns the “best” one

  transcript = " ".join(chunk.text for chunk in transcript_list) # Flatten it to plain text
  print(transcript)
except TranscriptsDisabled:
  print("No captions available for this video.")

Documentary In the north of India, in the city of Agra, 
stands the Taj Mahal, a monumental mausoleum   of white marble built almost 400 years ago on the 
banks of the sacred Yamuna River. I don't think a   building like this will ever be created again. The 
dome of this mythical building towers 73 meters,   marking the heart of a vast architectural complex 
stretching 177,000 square meters. It's absolutely   grand in scale. So the Taj can really be seen 
from the moon or from two centimeters away,   and it's perfect. The Taj Mahal is a symbol 
of perfection. The Mughal emperor Shah Jahan   ordered it built in the 17th century as a monument 
of love, in memory of his wife Mumtaz Mahal. Today   it is one of the ten most visited monuments in 
the world with nearly 7 million visitors each   year. The extraordinary mausoleum took builders 
just 18 years to complete and is considered one   of the seven wonders of the modern world. It 
is a masterpiece of architecture and I think   it was bu

In [7]:
transcript_list

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Documentary', start=0.0, duration=1.0), FetchedTranscriptSnippet(text='In the north of India, in the city of Agra,\xa0\nstands the Taj Mahal, a monumental mausoleum\xa0\xa0', start=1.0, duration=7.16), FetchedTranscriptSnippet(text="of white marble built almost 400 years ago on the\xa0\nbanks of the sacred Yamuna River. I don't think a\xa0\xa0", start=8.16, duration=6.92), FetchedTranscriptSnippet(text='building like this will ever be created again. The\xa0\ndome of this mythical building towers 73 meters,\xa0\xa0', start=15.08, duration=7.12), FetchedTranscriptSnippet(text="marking the heart of a vast architectural complex\xa0\nstretching 177,000 square meters. It's absolutely\xa0\xa0", start=22.2, duration=8.0), FetchedTranscriptSnippet(text='grand in scale. So the Taj can really be seen\xa0\nfrom the moon or from two centimeters away,\xa0\xa0', start=30.2, duration=5.04), FetchedTranscriptSnippet(text="and it's perfect. The 

**Step 1b - Indexing (Text Splitting)**

In [8]:
#create splitter object for chunking and initializing
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([transcript])

In [9]:
len(chunks)

53

In [10]:
chunks[0]

Document(metadata={}, page_content="Documentary In the north of India, in the city of Agra,\xa0\nstands the Taj Mahal, a monumental mausoleum\xa0\xa0 of white marble built almost 400 years ago on the\xa0\nbanks of the sacred Yamuna River. I don't think a\xa0\xa0 building like this will ever be created again. The\xa0\ndome of this mythical building towers 73 meters,\xa0\xa0 marking the heart of a vast architectural complex\xa0\nstretching 177,000 square meters. It's absolutely\xa0\xa0 grand in scale. So the Taj can really be seen\xa0\nfrom the moon or from two centimeters away,\xa0\xa0 and it's perfect. The Taj Mahal is a symbol\xa0\nof perfection. The Mughal emperor Shah Jahan\xa0\xa0 ordered it built in the 17th century as a monument\xa0\nof love, in memory of his wife Mumtaz Mahal. Today\xa0\xa0 it is one of the ten most visited monuments in\xa0\nthe world with nearly 7 million visitors each\xa0\xa0 year. The extraordinary mausoleum took builders\xa0\njust 18 years to complete and is

**Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)**

In [11]:
#create embedding and vector objects and initializing
embeddings = GoogleGenerativeAIEmbeddings(model = "models/gemini-embedding-001")
vector_store = FAISS.from_documents(chunks, embeddings)

In [12]:
#map vectors to documents
vector_store.index_to_docstore_id

{0: '9909858a-fdff-4de6-8173-7242135e48fa',
 1: 'efdcf569-ed17-4e7b-9d4f-71b774f223a1',
 2: 'c90a56d5-dda4-4fbd-980e-015225439cc2',
 3: '0b06bfbe-f609-4ca2-9ace-d9866841fbc5',
 4: '97090c61-6bb4-4187-bfeb-f23e80e4f2f3',
 5: '6fb1e1aa-634d-4cdf-aa64-d0586cc02430',
 6: '66ec79f5-255d-4e1c-a545-6db8423aa59f',
 7: 'da74aaf2-a66b-4b71-8691-af480b7a1aa5',
 8: 'b7a53a3f-d1ae-4922-948e-c950bc4374e0',
 9: 'e228cca7-44eb-4621-beb4-fc4964c08f89',
 10: '7b77dd12-c6b4-4c81-bc14-d807b8f31836',
 11: '63bace6f-762e-4307-ae0d-aad0aeb508b5',
 12: '08637560-e03b-4493-878c-533a40b46cad',
 13: 'b5c0fa6b-a19c-4049-bbea-77138313c22d',
 14: '8e2a5a9c-170f-4650-a496-d8a922d3295f',
 15: '9e50913b-8f12-411c-9421-08876203f1a1',
 16: '69703824-2dac-4a8f-a610-6cd707122e7a',
 17: '0f3c274c-b2df-442d-af0c-d2b0ab0d8c0c',
 18: '0c13a299-5638-46bd-8903-b830ffc361f5',
 19: 'af36023d-1827-4c6a-875e-ed05d22bf754',
 20: '85f83fe7-33a8-4e13-aeaa-5e0c99b2f13d',
 21: 'a22cea2a-4a72-4cef-9926-a8262fea7be2',
 22: '93c3c75a-5f5b-

In [13]:
#fectch document through vector id
vector_store.get_by_ids(['99a95e3e-0f46-4004-b83e-fdb1e00033c3'])

[]

**Step 2 - Retrieval**

In [14]:
#create a retriever object
retriever =  vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 4})

In [15]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7bb5f13a2270>, search_kwargs={'k': 4})

In [16]:
retriever.invoke("when taj mahal was built?")

[Document(id='c46f8a89-38a5-4867-a81e-1c12c8e54600', metadata={}, page_content="Mahal remains one of the most stable monuments\xa0\xa0 in the Indo-Islamic world thanks to revolutionary\xa0\nconstruction techniques. The Taj Mahal represents\xa0\xa0 an amalgamation of construction techniques\xa0\nwhich do not only come from India. They are\xa0\xa0 also imported from Central Asia, from the Iranian\xa0\nworld. These are notably used in the construction\xa0\xa0 of military works, forts and also dams. The\xa0\nfoundations of the mausoleum are inspired by\xa0\xa0 military architecture and civil engineering and\xa0\nare intended to be indestructible. They represent\xa0\xa0 the most successful construction techniques of\xa0\nAsia at the time of the Emperor Shah Jahan. The\xa0\xa0 Taj Mahal was built by the Emperor Shah Jahan\xa0\n400 years ago, after the death of his wife.\xa0\xa0 Today it's a symbol of beauty. Millions of\xa0\nvisitors come every year to admire this icon\xa0\xa0 of perfection.

**Step 3 - Augmentation**

In [17]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash", temperature = 0.2)

In [18]:
prompt = PromptTemplate(
    template = """You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.
      Context: {context}
      Question: {question}
      """,
    input_variables = ["context", "question"]
)

In [19]:
question = "explain the video summary in 5 lines and start each line with numbers sequence?"
retrieved_docs = retriever.invoke(question)

In [20]:
retrieved_docs

[Document(id='463e8a47-8c79-4e03-af4c-bb6587caf1ad', metadata={}, page_content="And so it's like a teaser. And once you get to\xa0\xa0 that entrance, suddenly it really comes up upon\xa0\nyou. A symbolic passage from the terrestrial world\xa0\xa0 to the celestial. Inside the final enclosure,\xa0\nsatellite images upon you. A symbolic passage from\xa0\xa0 the terrestrial world to the celestial. Inside the\xa0\nfinal enclosure, satellite images reveal that the\xa0\xa0 garden is divided by four water channels. The Holy\xa0\nQuran repeatedly promises the faithful gardens\xa0\xa0 beneath which rivers flow as their final resting\xa0\nspace. So paradise is really defined by the four\xa0\xa0 rivers of paradise, but that is most evident at\xa0\nthe Taj Mahal, where the water flowing represents\xa0\xa0 the four rivers of paradise. The mausoleum itself\xa0\nis an octagon, a significant choice. The Holy\xa0\xa0 Quran talks about the eight gateways of paradise,\xa0\nand the central tomb chamber at 

In [21]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"And so it's like a teaser. And once you get to\xa0\xa0 that entrance, suddenly it really comes up upon\xa0\nyou. A symbolic passage from the terrestrial world\xa0\xa0 to the celestial. Inside the final enclosure,\xa0\nsatellite images upon you. A symbolic passage from\xa0\xa0 the terrestrial world to the celestial. Inside the\xa0\nfinal enclosure, satellite images reveal that the\xa0\xa0 garden is divided by four water channels. The Holy\xa0\nQuran repeatedly promises the faithful gardens\xa0\xa0 beneath which rivers flow as their final resting\xa0\nspace. So paradise is really defined by the four\xa0\xa0 rivers of paradise, but that is most evident at\xa0\nthe Taj Mahal, where the water flowing represents\xa0\xa0 the four rivers of paradise. The mausoleum itself\xa0\nis an octagon, a significant choice. The Holy\xa0\xa0 Quran talks about the eight gateways of paradise,\xa0\nand the central tomb chamber at the Taj Mahal is\xa0\xa0 surrounded by eight chambers, which represent the\n\ni

In [22]:
#invoke prompt
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n      Context: And so it's like a teaser. And once you get to\xa0\xa0 that entrance, suddenly it really comes up upon\xa0\nyou. A symbolic passage from the terrestrial world\xa0\xa0 to the celestial. Inside the final enclosure,\xa0\nsatellite images upon you. A symbolic passage from\xa0\xa0 the terrestrial world to the celestial. Inside the\xa0\nfinal enclosure, satellite images reveal that the\xa0\xa0 garden is divided by four water channels. The Holy\xa0\nQuran repeatedly promises the faithful gardens\xa0\xa0 beneath which rivers flow as their final resting\xa0\nspace. So paradise is really defined by the four\xa0\xa0 rivers of paradise, but that is most evident at\xa0\nthe Taj Mahal, where the water flowing represents\xa0\xa0 the four rivers of paradise. The mausoleum itself\xa0\nis an octagon, a significant ch

**Step 4 - Generation**

In [23]:
#invoke model
answer = llm.invoke(final_prompt)
print(answer.content)

1.  The Taj Mahal is designed as a symbolic passage from the terrestrial world to the celestial, with its garden divided by four water channels representing the rivers of paradise.
2.  The mausoleum's octagonal shape and eight surrounding chambers symbolize the eight gateways of paradise.
3.  Satellite images and new technology allow for reinterpretation of the monument's layout and reveal clues about the site's evolution.
4.  Microscopic study of the mortar reveals surprising ingredients like plant debris, seeds, sugar, spices, chickpea flour, local fruit, and tree resin.
5.  These natural materials in the mortar have highly impermeable properties, contributing to the Taj Mahal's longevity.


**Building a Chain**

In [24]:
#import libraries
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

In [25]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [26]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [27]:
# parallel_chain.invoke("who is dhruv rathee")

In [28]:
parser = StrOutputParser()

In [29]:
main_chain = parallel_chain | prompt | llm | parser

In [30]:
result = main_chain.invoke("tell me the summary of this video?")
print(result)

The Taj Mahal's layout and construction are examined using satellite images and microscopic analysis. The satellite images reveal a pattern south of the main entrance, and the Taj Mahal is at the north end of the layout. The analysis of the mortar reveals it's made of minerals, plants, seeds, and possibly spices, like pepper. The ingredients have highly impermeable properties.
